# Graph-Based Modeling of a School Social Network


## Assignment 1

### Part 1: Build the Friendship Network

In [1]:
import random
import heapq
from collections import defaultdict

# ---------- Part 1: Generate Friendship Graph ----------
def generate_graph(num_students, num_classes=3):
    graph = defaultdict(list)

    # Assign each student to a class
    class_map = {i: i % num_classes for i in range(num_students)}

    # Assign popularity score to each student (higher = more likely to make friends)
    popularity = {i: random.randint(1, 5) for i in range(num_students)}  # 1 to 5

    # Initial friendship generation with bias
    for i in range(num_students):
        for j in range(i + 1, num_students):
            # Base probability
            p = 0.1

            # Intra-class: increase chance
            if class_map[i] == class_map[j]:
                p += 0.3

            # More popular students make more connections
            p += 0.05 * popularity[i]

            # Make connection?
            if random.random() < p:
                weight = random.randint(1, 10)
                graph[i].append((j, weight))
                graph[j].append((i, weight))

    # Add clustering: friends-of-friends more likely to connect
    for student in range(num_students):
        direct_friends = [n for n, _ in graph[student]]
        for friend in direct_friends:
            for ff, _ in graph[friend]:
                if ff != student:
                    already_friends = any(n == ff for n, _ in graph[student])
                    if not already_friends and random.random() < 0.2:
                        weight = random.randint(1, 10)
                        graph[student].append((ff, weight))
                        graph[ff].append((student, weight))

    return graph, class_map

### Part 2: Analyze Friend Groups

In [2]:
# ---------- Part 2: DFS and Group Analysis ----------
def dfs(graph, start, visited):
    stack = [start]
    group = []
    while stack:
        node = stack.pop()
        if not visited[node]:
            visited[node] = True
            group.append(node)
            for neighbor, _ in graph[node]:
                stack.append(neighbor)
    return group

def analyze_components(graph, num_nodes):
    visited = {}
    components = []

    for node in graph:
        visited[node] = False

    for node in graph:
        if not visited[node]:
            group = dfs(graph, node, visited)
            components.append(group)

    sizes = [len(c) for c in components]
    return components, min(sizes) if sizes else 0, max(sizes) if sizes else 0

### Part 3: Find Closest Friendship Paths

In [3]:
# ---------- Part 3: Dijkstra and A* ----------
def dijkstra(graph, start, end):
    heap = [(0, start, [])]
    visited = set()

    while heap:
        cost, node, path = heapq.heappop(heap)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return path, cost
        visited.add(node)
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(heap, (cost + weight, neighbor, path))
    return None, float('inf')

def a_star(graph, start, end, class_map):
    heap = [(0, 0, start, [])]
    visited = set()

    while heap:
        est, cost, node, path = heapq.heappop(heap)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return path, cost
        visited.add(node)
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                h = 0 if class_map[neighbor] == class_map[end] else 5
                heapq.heappush(heap, (cost + weight + h, cost + weight, neighbor, path))
    return None, float('inf')

### Bonus: Detect Social Bridges

In [4]:
# ---------- Bonus: Bridge Detection ----------
def find_bridges(graph, num_nodes):
    original_components, _, _ = analyze_components(graph, num_nodes)
    bridge_nodes = []

    for node in range(num_nodes):
        # Deep copy and remove node
        modified_graph = defaultdict(list)
        for k, v in graph.items():
            if k == node:
                continue
            modified_graph[k] = [(n, w) for n, w in v if n != node]

        components, _, _ = analyze_components(modified_graph, num_nodes)
        if len(components) > len(original_components):
            bridge_nodes.append(node)

    return bridge_nodes

In [5]:
# ---------- Main Execution ----------
def main():
    num_students = 150
    graph, class_map = generate_graph(num_students)

    print("\n📊 Analyzing Friend Groups...")
    components, smallest, largest = analyze_components(graph, num_students)
    print(f"🔹 Number of groups: {len(components)}")
    print(f"🔸 Smallest group size: {smallest}, Largest: {largest}")

    print("\n🛣️ Finding Shortest Paths...")
    pairs = random.sample(range(num_students), 10)
    for i in range(0, len(pairs), 2):
        u, v = pairs[i], pairs[i + 1]
        path_d, cost_d = dijkstra(graph, u, v)
        path_a, cost_a = a_star(graph, u, v, class_map)
        print(f"\n{u} ➡ {v}")
        print(f"  Dijkstra: Path={path_d}, Cost={cost_d}")
        print(f"  A*:       Path={path_a}, Cost={cost_a}")

    print("\n🌉 Detecting Bridge Nodes...")
    bridges = find_bridges(graph, num_students)
    print(f"Bridge Nodes: {bridges}")

if __name__ == "__main__":
    main()



📊 Analyzing Friend Groups...
🔹 Number of groups: 1
🔸 Smallest group size: 150, Largest: 150

🛣️ Finding Shortest Paths...

87 ➡ 88
  Dijkstra: Path=[87, 32, 88], Cost=2
  A*:       Path=[87, 28, 88], Cost=4

39 ➡ 43
  Dijkstra: Path=[39, 47, 43], Cost=2
  A*:       Path=[39, 4, 43], Cost=3

42 ➡ 97
  Dijkstra: Path=[42, 48, 97], Cost=2
  A*:       Path=[42, 37, 97], Cost=3

131 ➡ 76
  Dijkstra: Path=[131, 94, 76], Cost=2
  A*:       Path=[131, 94, 76], Cost=2

59 ➡ 137
  Dijkstra: Path=[59, 53, 137], Cost=2
  A*:       Path=[59, 53, 137], Cost=2

🌉 Detecting Bridge Nodes...
Bridge Nodes: []
